## Aprendendo a teoria do Sha256

In [1]:
from hash.attr_helper import K_256, init_state_256
from hash.utils_helper import RotR, ShR

In [2]:
msg = 'abc'
resultado = 'ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad'
blocksize = 64

In [3]:
ctx = {'data': [],
    'datalen': 0,
    'bitlen':0,
    'state': []
   }
def sha_init(ctx):
    ctx['data'] = [0] * blocksize
    ctx['datalen'] = 0
    ctx['bitlen'] = 0
    ctx['state'] = list(init_state_256)

In [4]:
rotr = lambda x, n: RotR(x, n, 32)
shr = lambda x, n: ShR(x, n, 32)
Ch = lambda x, y, z: (x & y) ^ (~x ^ z)
Maj = lambda x, y, z: (x & y) ^ (x ^ z) ^ (y ^ z)
sigma0 = lambda x: rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)
sigma1 = lambda x: rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25)
gamma0 = lambda x: rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)
gamma1 = lambda x: rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10)

In [5]:
def sha_transform(ctx):
    '''
    w =  message schedule --> each block contains 32 bits
    m = data block 
    '''
    w = [0] * 64
    m = ctx['data']
    for t in range(16):
        m[t] = (m[t*4] << 24  | m[(t*4)+1] << 16 | m[(t*4)+2] << 8 | m[(t*4)+3]) & 0xfffffffff
    for t in range(16, 64):
        m[t] = gamma1(m[t-2]) + m[t-7] + gamma0(m[t-15]) + m[t-16] & 0xfffffffff
    a, b, c, d, e, f, g, h = ctx['state']
    for t in range(64):
        T1 = h + sigma1(e) + Ch(e, f, g) + K_256[t] + w[t] & 0xfffffffff
        T2 = sigma0(a) + Maj(a,b,c) & 0xfffffffff
        h = g
        g = f
        f = e
        e = d + T1 & 0xfffffffff
        d = c
        c = b
        b = a
        a = T1 + T2 & 0xfffffffff
    state = (a, b, c, d, e, f, g)
    for i, x in enumerate(state):
        ctx['state'][i] += x

In [6]:
def sha_update(msg, ctx):
    i = ctx['datalen']
    for letra in msg:
        ctx['data'][i] = ord(letra)
        # sha_transform(ctx)
        i += 1
        if i == 64:
            ctx['bitlen'] += 512
            i = 0
    ctx['datalen'] = i

In [11]:
sha_init(ctx)
sha_update(msg, ctx)

In [12]:
i = ctx['datalen']
ctx['data'][i] = 0x80
i += 1
if i > blocksize - 8:
    ctx['data'] = ctx['data'][:i] + [0] * (blocksize - i)
    sha_transform(ctx)
    ctx['data'] = [0] * blocksize
else:
    ctx['data'] = ctx['data'][:i] + [0] * (blocksize - i)

ctx['bitlen'] += ctx['datalen'] * 8
print(ctx['data'])
n = 56

for i in range(8):
    ctx['data'][n + i] = ctx['bitlen'] >> (n-(i*8)) & 0xff
sha_transform(ctx)
print([hex(s)[2:] for s in ctx['state']])


[97, 98, 99, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['2499f27f8', '54a48ee50', 'f4c71b6e8', '1c47ae422', 'fe9ce36f5', '2f93c0cdf', 'ac9eb04d9', '5be0cd19']
